In [ ]:
#%pip install pandas
#%pip install kaggle
#%pip install openpyxl

# import pandas library
import pandas as pd

# import kaggle library for downloading datasets
import kaggle

#import zipfile library to extract zip from kaggle
import zipfile

#import openpyxl to write to excel
import openpyxl


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\jackn\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.9 kB ? eta -:--:--
   ---------------------- ----------------- 143.4/250.9 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
# download dataset from Kaggle
!kaggle datasets download -d hmavrodiev/london-bike-sharing-dataset

Dataset URL: https://www.kaggle.com/datasets/hmavrodiev/london-bike-sharing-dataset
License(s): other
london-bike-sharing-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
# unzip the downloaded dataset
zipfile_name = 'london-bike-sharing-dataset.zip'
with zipfile.ZipFile(zipfile_name, 'r') as file:
    file.extractall()

In [16]:
# read the dataset into a pandas dataframe
bikes = pd.read_csv('london_merged.csv')

In [5]:
# explore the dataset
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     17414 non-null  object 
 1   cnt           17414 non-null  int64  
 2   t1            17414 non-null  float64
 3   t2            17414 non-null  float64
 4   hum           17414 non-null  float64
 5   wind_speed    17414 non-null  float64
 6   weather_code  17414 non-null  float64
 7   is_holiday    17414 non-null  float64
 8   is_weekend    17414 non-null  float64
 9   season        17414 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.3+ MB


In [6]:
bikes.shape

(17414, 10)

In [21]:
bikes.head()

time  count  temp_real_C  temp_feel_C  humidity  \
0  2015-01-04 00:00:00    182          3.0          2.0     0.930   
1  2015-01-04 01:00:00    138          3.0          2.5     0.930   
2  2015-01-04 02:00:00    134          2.5          2.5     0.965   
3  2015-01-04 03:00:00     72          2.0          2.0     1.000   
4  2015-01-04 04:00:00     47          2.0          0.0     0.930   

   wind_speed_kmh  weather  is_holiday  is_weekend  season  
0             6.0      3.0         0.0         1.0     3.0  
1             5.0      1.0         0.0         1.0     3.0  
2             0.0      1.0         0.0         1.0     3.0  
3             0.0      1.0         0.0         1.0     3.0  
4             6.5      1.0         0.0         1.0     3.0

In [8]:
# count unique values in weather_code column
bikes.weather_code.value_counts()

weather_code
1.0     6150
2.0     4034
3.0     3551
7.0     2141
4.0     1464
26.0      60
10.0      14
Name: count, dtype: int64

In [9]:
# count unique values in season column
bikes.season.value_counts()

season
0.0    4394
1.0    4387
3.0    4330
2.0    4303
Name: count, dtype: int64

In [18]:
# create a dictionary to rename columns
new_cols_dict = {
    'timestamp' : 'time',
    'cnt' : 'count',
    't1' : 'temp_real_C',
    't2' : 'temp_feel_C',
    'hum' : 'humidity',
    'wind_speed' : 'wind_speed_kmh',
    'weather_code' : 'weather',
    'is_holiday' : 'is_holiday',
    'is_weekend' : 'is_weekend',
    'season' : 'season'
    }

# rename columns using the dictionary above
bikes.rename(columns=new_cols_dict, inplace=True)

In [20]:
# change humidity column to percentage
bikes.humidity = bikes.humidity / 100

In [22]:
# create a dictionary to map season values to names
season_dict = {
    '0.0' : 'spring',
    '1.0' : 'summer',
    '2.0' : 'autumn',
    '3.0' : 'winter'
}

# create a dictionary to map weather values to names
weather_dict = {
    '1.0' : 'Clear',
    '2.0' : 'Scattered clouds',
    '3.0' : 'Broken clouds',
    '4.0' : 'Cloudy',
    '7.0' : 'Rain',
    '10.0' : 'Rain with thunderstorm',
    '26.0' : 'Snowfall'
}

# change season types to string for mapping
bikes.season = bikes.season.astype(str)
# map season and weather columns to names
bikes.season = bikes.season.map(season_dict)

# change weather types to string for mapping
bikes.weather = bikes.weather.astype(str)
# map weather column to names
bikes.weather = bikes.weather.map(weather_dict)

In [23]:
# checking the changes
bikes.head()

time  count  temp_real_C  temp_feel_C  humidity  \
0  2015-01-04 00:00:00    182          3.0          2.0     0.930   
1  2015-01-04 01:00:00    138          3.0          2.5     0.930   
2  2015-01-04 02:00:00    134          2.5          2.5     0.965   
3  2015-01-04 03:00:00     72          2.0          2.0     1.000   
4  2015-01-04 04:00:00     47          2.0          0.0     0.930   

   wind_speed_kmh        weather  is_holiday  is_weekend  season  
0             6.0  Broken clouds         0.0         1.0  winter  
1             5.0          Clear         0.0         1.0  winter  
2             0.0          Clear         0.0         1.0  winter  
3             0.0          Clear         0.0         1.0  winter  
4             6.5          Clear         0.0         1.0  winter

In [26]:
# writing the cleaned dataset to an excel file for use with Tableau
bikes.to_excel('london_bikes_cleaned.xlsx', sheet_name='Data')